In [3]:
import pandas as pd

Import CSV

In [4]:
eshop = pd.read_csv('Sample-Superstore.csv', encoding='latin1')

In [3]:
display(eshop.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

None

In [5]:
eshop.columns = [col.lower().replace(' ', '_') for col in eshop.columns]



In [6]:
eshop.rename({'sub-category': 'sub_category'}, axis='columns', inplace=True)


In [7]:
eshop[['first_name', 'surname']] = eshop['customer_name'].str.split(' ', n=1, expand=True)



In [7]:
display(eshop.head())

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,first_name,surname
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,Claire,Gute
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,Claire,Gute
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,Darrin,Van Huff
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,Sean,O'Donnell
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,Sean,O'Donnell


In [8]:
eshop['order_date'] = pd.to_datetime(eshop['order_date']).dt.strftime('%Y-%m-%d')


In [9]:
import os
import sqlite3
from sqlite3 import Error

In [11]:
eshop.to_csv('Sample-Superstore3.csv', index=False)

In [10]:
path = r'C:\\Asus WebStorage\\psabin@gmail.com\\MySyncFolder\\Data Science Course\\E-Commerce\\'

In [11]:
eshop_db = sqlite3.connect('eshop_database.db')

In [12]:
c = eshop_db.cursor()

In [37]:
c.execute('''
CREATE TABLE IF NOT EXISTS "orders" (
	"order_id"	TEXT,
	"order_date"	TEXT,
	"ship_date"	TEXT,
	"ship_mode"	TEXT,
	"customer_id"	TEXT,
	"address_id"	TEXT,
	PRIMARY KEY("order_id")
);
''')

In [35]:
c.execute('''
CREATE TABLE IF NOT EXISTS "products" (
	"product_id"	TEXT,
	"product_name"	TEXT,
	"category"	TEXT,
	"sub_category"	TEXT,
	PRIMARY KEY("product_id")
);
''')

In [38]:
c.execute('''
CREATE TABLE IF NOT EXISTS "customers" (
	"customer_id"	TEXT,
	"first_name"	TEXT,
	"surname"	TEXT,
	"segment"	TEXT,
	PRIMARY KEY("customer_id")
);
''')

In [40]:
c.execute('''
CREATE TABLE IF NOT EXISTS "addresses" (
	"address_id"	TEXT,
	"country"	TEXT,
	"city"	TEXT,
	"state"	TEXT,	
    "postal_code"	TEXT,
	"region"	TEXT,
	PRIMARY KEY("address_id")
);
''')

In [13]:
# Create a new DataFrame for addresses with unique combinations
addresses = eshop.drop_duplicates(subset=['customer_id', 'country', 'city', 'state', 'postal_code'])

# Sort by customer_id to maintain order
addresses.sort_values(by='customer_id', inplace=True)

# Generate a unique address_id by enumerating customer addresses
addresses['address_id'] = addresses.groupby('customer_id').cumcount() + 1
addresses['address_id'] = addresses['customer_id'] + 'A' + addresses['address_id'].astype(str)

display(addresses.head())


C:\Users\psabi\AppData\Local\Temp\ipykernel_26860\686817426.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses.sort_values(by='customer_id', inplace=True)
C:\Users\psabi\AppData\Local\Temp\ipykernel_26860\686817426.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses['address_id'] = addresses.groupby('customer_id').cumcount() + 1
C:\Users\psabi\AppData\Local\Temp\ipykernel_26860\686817426.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,category,sub_category,product_name,sales,quantity,discount,profit,first_name,surname,address_id
1159,1160,CA-2017-147039,2017-06-29,7/4/2017,Standard Class,AA-10315,Alex Avila,Consumer,United States,Minneapolis,...,Office Supplies,Appliances,"Belkin 325VA UPS Surge Protector, 6'",362.940,3,0.0,90.7350,Alex,Avila,AA-10315A1
1299,1300,CA-2015-121391,2015-10-04,10/7/2015,First Class,AA-10315,Alex Avila,Consumer,United States,San Francisco,...,Office Supplies,Storage,Tenex Personal Project File with Scoop Front D...,26.960,2,0.0,7.0096,Alex,Avila,AA-10315A2
5198,5199,CA-2016-103982,2016-03-03,3/8/2016,Standard Class,AA-10315,Alex Avila,Consumer,United States,Round Rock,...,Office Supplies,Supplies,High Speed Automatic Electric Letter Opener,3930.072,3,0.2,-786.0144,Alex,Avila,AA-10315A3
7468,7469,CA-2014-138100,2014-09-15,9/20/2014,Standard Class,AA-10315,Alex Avila,Consumer,United States,New York City,...,Office Supplies,Paper,Easy-staple paper,14.940,3,0.0,7.0218,Alex,Avila,AA-10315A4
2229,2230,CA-2014-128055,2014-03-31,4/5/2014,Standard Class,AA-10315,Alex Avila,Consumer,United States,San Francisco,...,Office Supplies,Binders,GBC DocuBind 200 Manual Binding Machine,673.568,2,0.2,252.5880,Alex,Avila,AA-10315A5


In [14]:
# Merge this 'address_id' back into the original DataFrame
eshop = eshop.merge(addresses[['customer_id', 'country', 'city', 'state', 'postal_code', 'address_id']], 
                    on=['customer_id', 'country', 'city', 'state', 'postal_code'], 
                    how='left')

In [ ]:
# Select only the columns that exist in the 'orders' table
orders_columns = ['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'address_id']  
orders_df = eshop[orders_columns]




In [54]:
orders_df_unique = orders_df.drop_duplicates()
display(orders_df_unique.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5009 entries, 0 to 9993
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     5009 non-null   object
 1   order_date   5009 non-null   object
 2   ship_date    5009 non-null   object
 3   ship_mode    5009 non-null   object
 4   customer_id  5009 non-null   object
 5   address_id   5009 non-null   object
dtypes: object(6)
memory usage: 273.9+ KB


None

In [55]:
# Import the filtered data into the database
orders_df_unique.to_sql('orders', eshop_db, if_exists='append', index=False)

5009

In [ ]:
# Select only the columns that exist in the 'orders' table
addresses_columns = ['address_id', 'country', 'city', 'state', 'postal_code', 'region']  
addresses_df = eshop[addresses_columns]
addresses_df_unique = addresses_df.drop_duplicates()
display(addresses_df.info())
display(addresses_df_unique.info())

In [57]:
addresses_df_unique.to_sql('addresses', eshop_db, if_exists='append', index=False)

4910

In [ ]:
# Select only the columns that exist in the 'orders' table
customers_columns = ['customer_id', 'first_name', 'surname', 'segment']  
customers_df = eshop[customers_columns]
customers_df_unique = customers_df.drop_duplicates()
display(customers_df.info())
display(customers_df_unique.info())

In [59]:
customers_df_unique.to_sql('customers', eshop_db, if_exists='append', index=False)

793

In [18]:

details_columns = ['row_id', 'order_id', 'product_id', 'quantity', 'discount', 'sales', 'profit']  
details_df = eshop[details_columns]
details_df_unique = details_df.drop_duplicates()
display(details_df_unique.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   row_id      9994 non-null   int64  
 1   order_id    9994 non-null   object 
 2   product_id  9994 non-null   object 
 3   quantity    9994 non-null   int64  
 4   discount    9994 non-null   float64
 5   sales       9994 non-null   float64
 6   profit      9994 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 546.7+ KB


None

In [22]:
details_df.to_sql('order_details', eshop_db, if_exists='append', index=False)

9994

In [20]:
c.execute('''
DROP TABLE IF EXISTS "order_details";
''')

In [21]:
c.execute('''
CREATE TABLE IF NOT EXISTS "order_details" (
    "row_id"	INTEGER,      
	"order_id"	TEXT,
	"product_id"	TEXT,
	"quantity"	INTEGER,
	"discount"	REAL,
	"sales"	REAL,
	"profit"	REAL,
	PRIMARY KEY("row_id")
);
''')

In [34]:
products_columns = ['product_id', 'product_name', 'category', 'sub_category']  
products_df = eshop[products_columns]
products_df_unique = products_df.drop_duplicates()

In [36]:
products_df_unique.to_sql('products', eshop_db, if_exists='append', index=False)

1894

In [35]:
# Finding duplicate rows based on 'product_id'
duplicate_rows = products_df_unique[products_df_unique.duplicated(subset=['product_id'], keep=False)]

# Displaying the duplicate rows
print(duplicate_rows)


Empty DataFrame
Columns: [product_id, product_name, category, sub_category]
Index: []


In [18]:
# Identify duplicates
duplicates = eshop[eshop.duplicated('product_id', keep=False)]


In [21]:

# Function to add suffixes
def add_suffix(sub_df):
    suffixes = [f'-{chr(65+i)}' for i in range(len(sub_df))]
    sub_df['product_id'] = sub_df['product_id'] + suffixes
    return sub_df

# Apply the function to each group of duplicates
unique_df = duplicate_rows.groupby('product_id').apply(add_suffix)



In [25]:
# Creating 'original_product_id' by slicing off the last two characters
unique_df['original_product_id'] = unique_df['product_id'].str[:-2]


In [26]:
# Create a mapping from the original product_id to the new one
product_id_mapping = unique_df.set_index('original_product_id')['product_id'].to_dict()

# Apply this mapping to the 'product_id' column in eshop
eshop['product_id'] = eshop['product_id'].map(product_id_mapping).fillna(eshop['product_id'])


In [30]:
unique_df['combined_key'] = unique_df['original_product_id'] + '_' + unique_df['product_name']


In [31]:
product_id_mapping = unique_df.set_index('combined_key')['product_id'].to_dict()


In [32]:
eshop['combined_key'] = eshop['product_id'].str[:-2] + '_' + eshop['product_name']
eshop['product_id'] = eshop['combined_key'].map(product_id_mapping).fillna(eshop['product_id'])
